# Simulation of Lennard-Jones Fluid #

This example will simulate a WCA fluid in HOOMD-blue and write a trajectory file of the simulation.

In [1]:
import relentless

Define a directory for the simulation to take place in. This can be an absolute or relative path.

In [2]:
dir = "lj_fluid_simulation/"

Define the $\sigma$ and $\epsilon$ parameters of the WCA potential.

In [3]:
sigma = 1
epsilon = 1

Set up the Lennard-Jones potential. Ensure that the potential is defined in terms of the design variables when applicable. Then attach the potential to the simulation using  `relentless.simulate.Potentials(pair_potentials=None, kB=1.0)`. The potential's starting point, stopping point, number of subdivisions, and buffer to be added to the end of the stopping point all must be defined. Note: The starting point should be greater than zero. 

In [4]:
lj = relentless.model.potential.LennardJones(types=("1",))
lj.coeff["1", "1"].update({
    "epsilon": epsilon, "sigma": sigma, "rmax": 3*sigma, "shift": True
})

pot = relentless.simulate.Potentials(lj)
pot.pair.start = 1e-6
pot.pair.stop = 3*sigma
pot.pair.num = 100
pot.pair.neighbor_buffer = 0.5

Initialize the system using `relentless.simulate.InitializeRandomly`. Relentless 2-dimensional and 3-dimensional boxes. 

`relentless.simulate.EnsembleAverage` analyzes the simulations thermodynamics properties and radial distribution function. 

`relentless.simulate.WriteTrajectory` writes a trajectory file of the simulation. 

The lgv block of code is the molecuar dynamics method to be ran. In this case Langevin dynamics is chosen; however, relentless supports several methods which could have been chosen instead. You can find more information about these in the documentation. 

In [5]:
init = relentless.simulate.InitializeRandomly(
    seed=42,
    T=1.5,
    V=relentless.model.extent.Cube(L=10.0),
    N={"1": 100},
    diameters={"1": 1}
)

anly = relentless.simulate.EnsembleAverage(
    check_thermo_every=10,
    check_rdf_every=100,
    rdf_dr=0.005
)

dump = relentless.simulate.WriteTrajectory(
    filename='lj_fluid_trajectory.gsd', 
    every=250
)

lgv = relentless.simulate.RunLangevinDynamics(
    steps=5000,
    timestep=0.005,
    T=1,
    friction=0.1,
    seed=2,
    analyzers=[anly, dump]
)

Run the simulation.

In [7]:
h = relentless.simulate.HOOMD(init, operations=[lgv])
sim = h.run(potentials=pot, directory=dir)